In [1]:
import json
import torch

In [3]:
from MyDataset import MyDataset
from torch.utils.data import DataLoader

In [ ]:
lama_embedding_path = '../llama_embedding/tumor_3.1_CL_finetuned/dataset_tensor/'
bert_embedding_path = '../bert_embedding/tumor_finetuned/dataset_tensor/'
roberta_embedding_path = '../roberta_embedding/tumor_finetuned/dataset_tensor/'

In [5]:
train_data = MyDataset('train', lama_embedding_path, bert_embedding_path, roberta_embedding_path)
test_data =MyDataset('test', lama_embedding_path, bert_embedding_path, roberta_embedding_path)

In [6]:
train_data = DataLoader(train_data, batch_size=1, shuffle=False)
test_data = DataLoader(test_data, batch_size=1, shuffle=False) 

In [7]:
for i, data in enumerate(train_data):
    print(data)
    break

[tensor([[[-1.5044,  0.6670,  2.3295,  ...,  0.9674, -2.3429,  1.8316],
         [-0.4019,  0.1248,  0.0830,  ...,  0.0686, -0.6041,  0.1625],
         [-0.4163,  0.2404,  0.1163,  ..., -0.1729, -0.4311,  0.2859],
         [-0.4265,  0.3218,  0.1691,  ..., -0.0411, -0.4049,  0.1611],
         [-0.2563,  0.5020,  0.2488,  ...,  0.0147, -0.3530,  0.1012]]]), tensor([[-0.0811, -0.7083,  0.8326,  ..., -0.6350,  0.9180,  0.5636]]), tensor([[-0.5352,  0.6397, -0.1189,  ..., -1.5615, -1.9301, -1.0474]]), tensor([3])]


In [8]:
l_rep, b_rep, r_rep, label = data

In [9]:
b_rep.squeeze().shape

torch.Size([1024])

In [10]:
l_rep.squeeze().shape, b_rep.squeeze().shape, r_rep.squeeze().shape, label.item()

(torch.Size([5, 4096]), torch.Size([1024]), torch.Size([1024]), 3)

In [11]:
# reps 별 코사인 유사도 평균 구해보기
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

In [12]:
#get_cosine_similarity(reps_dict[0]['lama'][0].flatten() , reps_dict[0]['lama'][1].flatten()).item()

# Label 끼리의 코사인 유사도
## Train Data

In [13]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [14]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [15]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.48030292987823486, std : 0.3244469463825226, min : -0.02666034922003746, max : 0.9885945320129395
Bert mean : 0.9927564263343811, std : 0.00888011883944273, min : 0.9261558651924133, max : 0.9999865889549255
Roberta mean : 0.9656540751457214, std : 0.08705567568540573, min : 0.37397775053977966, max : 0.9999938011169434

Label 1 :
Lama mean : 0.7042180299758911, std : 0.16205398738384247, min : -0.020111149176955223, max : 0.9940289855003357
Bert mean : 0.9966709017753601, std : 0.0028710165061056614, min : 0.9811343550682068, max : 0.9999918937683105
Roberta mean : 0.9827726483345032, std : 0.014596483670175076, min : 0.8483975529670715, max : 0.9997824430465698

Label 2 :
Lama mean : 0.5735873579978943, std : 0.2338789403438568, min : -0.04098382592201233, max : 0.9748389720916748
Bert mean : 0.9953146576881409, std : 0.008820051327347755, min : 0.9241839647293091, max : 1.0000001192092896
Roberta mean : 0.9897527098655701, std : 0.0062699466943740845, min : 0

## Test Data

In [16]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [17]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [18]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.4840444326400757, std : 0.3246239125728607, min : -0.015935935080051422, max : 0.9873759150505066
Bert mean : 0.8825313448905945, std : 0.3595038950443268, min : -0.2971953749656677, max : 0.9997621774673462
Roberta mean : 0.9556592106819153, std : 0.05133352428674698, min : 0.7699362635612488, max : 0.999946117401123

Label 1 :
Lama mean : 0.598544716835022, std : 0.2573408782482147, min : -0.023980149999260902, max : 0.9900539517402649
Bert mean : 0.6866618394851685, std : 0.4628823399543762, min : -0.3245903551578522, max : 0.999981701374054
Roberta mean : 0.6968095302581787, std : 0.39791131019592285, min : -0.018198715522885323, max : 0.9996711015701294

Label 2 :
Lama mean : 0.6018698215484619, std : 0.2203962653875351, min : 0.006546007469296455, max : 0.980207622051239
Bert mean : 0.9619156122207642, std : 0.06619256734848022, min : 0.7581995129585266, max : 0.9999444484710693
Roberta mean : 0.984171986579895, std : 0.021882247179746628, min : 0.89715433

## Llama Layer 별 코사인 유사도

### Train

In [33]:
reps_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    l_rep_0 = l_rep[0]
    l_rep_1 = l_rep[1]
    l_rep_2 = l_rep[2]
    l_rep_3 = l_rep[3]
    l_rep_4 = l_rep[4]
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'][0].append(l_rep_0)
    reps_dict[label]['lama'][1].append(l_rep_1)
    reps_dict[label]['lama'][2].append(l_rep_2)
    reps_dict[label]['lama'][3].append(l_rep_3)
    reps_dict[label]['lama'][4].append(l_rep_4)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)

cnt == len(train_data)

True

In [34]:
all_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
for idx in range(6):
    # bert, roberta 각각의 평균 코사인 유사도 구하기
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['bert'])):
        for j in range(i+1, len(reps_dict[idx]['bert'])):
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)
# llama layer별
for idx in range(6):
    for i in range(5): # lama layer별
        lama_cosine = []
        for j in range(len(reps_dict[idx]['lama'][i])):
            for k in range(j+1, len(reps_dict[idx]['lama'][i])):
                lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i][j].flatten(),reps_dict[idx]['lama'][i][k].flatten()).item())
        
        all_dict[idx]['lama'][i] = torch.tensor(lama_cosine)

In [21]:
all_dict

{0: {'lama': {0: tensor([ 0.6344,  0.1352,  0.8553,  ...,  0.6920,  0.0605, -0.0425]),
   1: tensor([0.6758, 0.5113, 0.8539,  ..., 0.7890, 0.3661, 0.2882]),
   2: tensor([0.7249, 0.5905, 0.8682,  ..., 0.8361, 0.3782, 0.3104]),
   3: tensor([0.7350, 0.6311, 0.8764,  ..., 0.8465, 0.3890, 0.3157]),
   4: tensor([0.7694, 0.6668, 0.8960,  ..., 0.8641, 0.3984, 0.3262])},
  'bert': tensor([0.9965, 0.9817, 0.9993,  ..., 0.9832, 0.9956, 0.9825]),
  'roberta': tensor([0.9552, 0.9433, 0.9975,  ..., 0.9304, 0.9637, 0.9434])},
 1: {'lama': {0: tensor([0.6794, 0.7393, 0.7902,  ..., 0.3354, 0.5700, 0.1227]),
   1: tensor([0.7381, 0.7872, 0.8148,  ..., 0.6435, 0.6945, 0.4575]),
   2: tensor([0.7814, 0.8233, 0.8422,  ..., 0.6785, 0.7457, 0.5053]),
   3: tensor([0.7903, 0.8447, 0.8545,  ..., 0.7105, 0.7683, 0.5502]),
   4: tensor([0.8157, 0.8703, 0.8731,  ..., 0.7335, 0.7970, 0.5872])},
  'bert': tensor([0.9982, 0.9992, 0.9981,  ..., 0.9944, 0.9976, 0.9978]),
  'roberta': tensor([0.9901, 0.9853, 0.9857,

In [22]:
reps_dict[idx]['lama']

{0: [tensor([ 0.9748,  1.0260,  3.0652,  ..., -0.2705,  0.3263,  1.3657]),
  tensor([ 0.0175,  2.6631,  0.5847,  ...,  0.8788, -0.1573,  1.0420]),
  tensor([ 1.2309,  1.1161,  3.4340,  ..., -0.6855,  1.3896,  1.5609]),
  tensor([-1.2121,  0.1060,  1.4803,  ..., -0.7845,  0.6384,  1.6443]),
  tensor([ 1.0451,  2.5450,  2.2916,  ..., -0.0995, -0.9983,  1.1426]),
  tensor([0.9411, 0.8431, 2.2509,  ..., 0.1797, 0.2007, 1.6376]),
  tensor([ 1.2576,  0.4515,  2.2577,  ..., -0.1496,  0.0136,  1.8808]),
  tensor([ 0.6587, -0.6355,  3.5464,  ...,  0.6583,  0.5341,  1.4443]),
  tensor([ 0.6783,  0.7595,  3.3668,  ..., -1.6063,  0.3123,  1.1846]),
  tensor([ 2.0099, -0.3052,  3.0834,  ...,  0.4188,  0.9428,  2.9046]),
  tensor([2.5103, 0.2459, 1.9508,  ..., 0.8245, 0.1666, 1.2920]),
  tensor([2.4977, 0.7005, 0.7694,  ..., 0.7291, 1.5053, 2.4429]),
  tensor([-0.8598,  0.3640,  1.9714,  ...,  1.4377, -0.6585,  0.7879]),
  tensor([-1.1287,  0.6013,  2.5646,  ...,  2.4176, -2.2848,  0.3262]),
  tenso

In [23]:
for i in range(6):
    print(f'Label {i} :')
    # lama layer 별
    print(f'lama layer 0  mean : {all_dict[i]["lama"][0].mean()} std : {all_dict[i]["lama"][0].std()} min : {all_dict[i]["lama"][0].min()} max : {all_dict[i]["lama"][0].max()}')
    print(f'lama layer 1  mean : {all_dict[i]["lama"][1].mean()} std : {all_dict[i]["lama"][1].std()} min : {all_dict[i]["lama"][1].min()} max : {all_dict[i]["lama"][1].max()}')
    print(f'lama layer 2  mean : {all_dict[i]["lama"][2].mean()} std : {all_dict[i]["lama"][2].std()} min : {all_dict[i]["lama"][2].min()} max : {all_dict[i]["lama"][2].max()}')
    print(f'lama layer 3  mean : {all_dict[i]["lama"][3].mean()} std : {all_dict[i]["lama"][3].std()} min : {all_dict[i]["lama"][3].min()} max : {all_dict[i]["lama"][3].max()}')
    print(f'lama layer 4  mean : {all_dict[i]["lama"][4].mean()} std : {all_dict[i]["lama"][4].std()} min : {all_dict[i]["lama"][4].min()} max : {all_dict[i]["lama"][4].max()}')


Label 0 :
lama layer 0  mean : 0.4582816958427429 std : 0.3399696946144104 min : -0.07265432178974152 max : 0.9879365563392639
lama layer 1  mean : 0.6002259850502014 std : 0.22470062971115112 min : 0.16175426542758942 max : 0.9912566542625427
lama layer 2  mean : 0.6332541704177856 std : 0.23044390976428986 min : 0.1818157434463501 max : 0.9924421310424805
lama layer 3  mean : 0.6474221348762512 std : 0.23154298961162567 min : 0.20032057166099548 max : 0.9934854507446289
lama layer 4  mean : 0.671459972858429 std : 0.23618336021900177 min : 0.21222372353076935 max : 0.9942880868911743
Label 1 :
lama layer 0  mean : 0.6929686665534973 std : 0.17042873799800873 min : -0.06605807691812515 max : 0.993717610836029
lama layer 1  mean : 0.7530477643013 std : 0.11401186883449554 min : 0.21396896243095398 max : 0.995693564414978
lama layer 2  mean : 0.7917009592056274 std : 0.10813746601343155 min : 0.25296857953071594 max : 0.9969007968902588
lama layer 3  mean : 0.8060942888259888 std : 0.10

### Test

In [24]:
reps_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    l_rep_0 = l_rep[0]
    l_rep_1 = l_rep[1]
    l_rep_2 = l_rep[2]
    l_rep_3 = l_rep[3]
    l_rep_4 = l_rep[4]
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'][0].append(l_rep_0)
    reps_dict[label]['lama'][1].append(l_rep_1)
    reps_dict[label]['lama'][2].append(l_rep_2)
    reps_dict[label]['lama'][3].append(l_rep_3)
    reps_dict[label]['lama'][4].append(l_rep_4)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)

cnt == len(test_data)

True

In [25]:
all_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
for idx in range(6):
    # bert, roberta 각각의 평균 코사인 유사도 구하기
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['bert'])):
        for j in range(i+1, len(reps_dict[idx]['bert'])):
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [26]:
for idx in range(6):
    for i in range(5): # lama layer별
        lama_cosine = []
        for j in range(len(reps_dict[idx]['lama'][i])):
            for k in range(j+1, len(reps_dict[idx]['lama'][i])):
                lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i][j].flatten(),reps_dict[idx]['lama'][i][k].flatten()).item())
        
        all_dict[idx]['lama'][i] = torch.tensor(lama_cosine)

In [27]:
for i in range(6):
    print(f'Label {i} :')
    # lama layer 별
    print(f'lama layer 0  mean : {all_dict[i]["lama"][0].mean()} std : {all_dict[i]["lama"][0].std()} min : {all_dict[i]["lama"][0].min()} max : {all_dict[i]["lama"][0].max()}')
    print(f'lama layer 1  mean : {all_dict[i]["lama"][1].mean()} std : {all_dict[i]["lama"][1].std()} min : {all_dict[i]["lama"][1].min()} max : {all_dict[i]["lama"][1].max()}')
    print(f'lama layer 2  mean : {all_dict[i]["lama"][2].mean()} std : {all_dict[i]["lama"][2].std()} min : {all_dict[i]["lama"][2].min()} max : {all_dict[i]["lama"][2].max()}')
    print(f'lama layer 3  mean : {all_dict[i]["lama"][3].mean()} std : {all_dict[i]["lama"][3].std()} min : {all_dict[i]["lama"][3].min()} max : {all_dict[i]["lama"][3].max()}')
    print(f'lama layer 4  mean : {all_dict[i]["lama"][4].mean()} std : {all_dict[i]["lama"][4].std()} min : {all_dict[i]["lama"][4].min()} max : {all_dict[i]["lama"][4].max()}')


Label 0 :
lama layer 0  mean : 0.46358874440193176 std : 0.3384522795677185 min : -0.05568278580904007 max : 0.986672580242157
lama layer 1  mean : 0.5983796119689941 std : 0.22708453238010406 min : 0.19598516821861267 max : 0.9901775121688843
lama layer 2  mean : 0.6308391094207764 std : 0.23406483232975006 min : 0.21928724646568298 max : 0.9917163252830505
lama layer 3  mean : 0.6414186954498291 std : 0.23933541774749756 min : 0.21745069324970245 max : 0.9927277565002441
lama layer 4  mean : 0.661725640296936 std : 0.2469824254512787 min : 0.24644210934638977 max : 0.9938218593597412
Label 1 :
lama layer 0  mean : 0.5795263648033142 std : 0.2719472348690033 min : -0.06521344184875488 max : 0.9896935224533081
lama layer 1  mean : 0.6882783770561218 std : 0.17085333168506622 min : 0.20340560376644135 max : 0.9917173981666565
lama layer 2  mean : 0.7320974469184875 std : 0.1680101603269577 min : 0.250400185585022 max : 0.9932606816291809
lama layer 3  mean : 0.7470207214355469 std : 0.1

# 다른 라벨들과의 코사인 유사도 비교
## TrainData

In [19]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [20]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [21]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.5413272976875305, std : 0.2847304940223694, min : -0.037676941603422165, max : 0.9191119074821472
Bert mean : -0.1666981279850006, std : 0.010526364669203758, min : -0.208282470703125, max : -0.12093579024076462
Roberta mean : -0.03852460905909538, std : 0.03408439829945564, min : -0.16758498549461365, max : 0.1900651603937149

Label 0 - Label 2 :
Lama mean : 0.20587420463562012, std : 0.26441818475723267, min : -0.09556057304143906, max : 0.9188210368156433
Bert mean : -0.2327321171760559, std : 0.0077667078003287315, min : -0.2909048795700073, max : -0.19608166813850403
Roberta mean : -0.018670227378606796, std : 0.0872267335653305, min : -0.09566497057676315, max : 0.8220071792602539

Label 0 - Label 3 :
Lama mean : 0.18722771108150482, std : 0.27942630648612976, min : -0.04657138139009476, max : 0.9261351823806763
Bert mean : -0.18996699154376984, std : 0.011919560842216015, min : -0.24039670825004578, max : -0.13743461668491364
Roberta mean : -0.0

### lama layer 별

### train

In [25]:
reps_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    l_rep_0 = l_rep[0]
    l_rep_1 = l_rep[1]
    l_rep_2 = l_rep[2]
    l_rep_3 = l_rep[3]
    l_rep_4 = l_rep[4]
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'][0].append(l_rep_0)
    reps_dict[label]['lama'][1].append(l_rep_1)
    reps_dict[label]['lama'][2].append(l_rep_2)
    reps_dict[label]['lama'][3].append(l_rep_3)
    reps_dict[label]['lama'][4].append(l_rep_4)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)

cnt == len(train_data)

True

In [26]:
all_dict = {i : {j :{'lama' : {k : [] for k in range(5)}}  for j in range(6) if i != j } for i in range(6)}
for main_label in range(6):
    for next_label in range(6):
        for layer_num in range(5):
            lama_cosine = []
            if main_label != next_label:
                for l in range(len(reps_dict[main_label]['lama'][layer_num])):
                    for k in range(len(reps_dict[next_label]['lama'][layer_num])):
                        lama_cosine.append(get_cosine_similarity(reps_dict[main_label]['lama'][layer_num][l], reps_dict[next_label]['lama'][layer_num][k]))
            
                all_dict[main_label][next_label]['lama'][layer_num] = torch.tensor(lama_cosine)

In [27]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} _ Layer : {0}')
            print(f'Lama mean : {all_dict[i][j]["lama"][0].mean()}, std : {all_dict[i][j]["lama"][0].std()}, min : {all_dict[i][j]["lama"][0].min()}, max : {all_dict[i][j]["lama"][0].max()}')
            
            print(f'Label {i} - Label {j} _ Layer : {1}')
            print(f'Lama mean : {all_dict[i][j]["lama"][1].mean()}, std : {all_dict[i][j]["lama"][1].std()}, min : {all_dict[i][j]["lama"][1].min()}, max : {all_dict[i][j]["lama"][1].max()}')

            print(f'Label {i} - Label {j} _ Layer : {2}')
            print(f'Lama mean : {all_dict[i][j]["lama"][2].mean()}, std : {all_dict[i][j]["lama"][2].std()}, min : {all_dict[i][j]["lama"][2].min()}, max : {all_dict[i][j]["lama"][2].max()}')

            print(f'Label {i} - Label {j} _ Layer : {3}')
            print(f'Lama mean : {all_dict[i][j]["lama"][3].mean()}, std : {all_dict[i][j]["lama"][3].std()}, min : {all_dict[i][j]["lama"][3].min()}, max : {all_dict[i][j]["lama"][3].max()}')

            print(f'Label {i} - Label {j} _ Layer : {4}')
            print(f'Lama mean : {all_dict[i][j]["lama"][4].mean()}, std : {all_dict[i][j]["lama"][4].std()}, min : {all_dict[i][j]["lama"][4].min()}, max : {all_dict[i][j]["lama"][4].max()}')


Label 0 - Label 1 _ Layer : 0
Lama mean : 0.5225430727005005, std : 0.2988731265068054, min : -0.07923213392496109, max : 0.9178649187088013
Label 0 - Label 1 _ Layer : 1
Lama mean : 0.6363747119903564, std : 0.19578397274017334, min : 0.16304539144039154, max : 0.9276597499847412
Label 0 - Label 1 _ Layer : 2
Lama mean : 0.6780866980552673, std : 0.19929613173007965, min : 0.18186984956264496, max : 0.9340721368789673
Label 0 - Label 1 _ Layer : 3
Lama mean : 0.6917426586151123, std : 0.20077604055404663, min : 0.1970122754573822, max : 0.9393244385719299
Label 0 - Label 1 _ Layer : 4
Lama mean : 0.7177930474281311, std : 0.2048761397600174, min : 0.21089527010917664, max : 0.9482427835464478
Label 0 - Label 2 _ Layer : 0
Lama mean : 0.16440613567829132, std : 0.2787114083766937, min : -0.19989082217216492, max : 0.9130476117134094
Label 0 - Label 2 _ Layer : 1
Lama mean : 0.4311179220676422, std : 0.1844501495361328, min : 0.17007513344287872, max : 0.9373192191123962
Label 0 - Label

# Test

In [28]:
reps_dict = {i : {'lama' : {j : [] for j in range(5)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    l_rep_0 = l_rep[0]
    l_rep_1 = l_rep[1]
    l_rep_2 = l_rep[2]
    l_rep_3 = l_rep[3]
    l_rep_4 = l_rep[4]
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'][0].append(l_rep_0)
    reps_dict[label]['lama'][1].append(l_rep_1)
    reps_dict[label]['lama'][2].append(l_rep_2)
    reps_dict[label]['lama'][3].append(l_rep_3)
    reps_dict[label]['lama'][4].append(l_rep_4)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)

cnt == len(test_data)

True

In [29]:
all_dict = {i : {j :{'lama' : {k : [] for k in range(5)}}  for j in range(6) if i != j } for i in range(6)}
for main_label in range(6):
    for next_label in range(6):
        for layer_num in range(5):
            lama_cosine = []
            if main_label != next_label:
                for l in range(len(reps_dict[main_label]['lama'][layer_num])):
                    for k in range(len(reps_dict[next_label]['lama'][layer_num])):
                        lama_cosine.append(get_cosine_similarity(reps_dict[main_label]['lama'][layer_num][l], reps_dict[next_label]['lama'][layer_num][k]))
            
                all_dict[main_label][next_label]['lama'][layer_num] = torch.tensor(lama_cosine)

In [30]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} _ Layer : {0}')
            print(f'Lama mean : {all_dict[i][j]["lama"][0].mean()}, std : {all_dict[i][j]["lama"][0].std()}, min : {all_dict[i][j]["lama"][0].min()}, max : {all_dict[i][j]["lama"][0].max()}')
            
            print(f'Label {i} - Label {j} _ Layer : {1}')
            print(f'Lama mean : {all_dict[i][j]["lama"][1].mean()}, std : {all_dict[i][j]["lama"][1].std()}, min : {all_dict[i][j]["lama"][1].min()}, max : {all_dict[i][j]["lama"][1].max()}')

            print(f'Label {i} - Label {j} _ Layer : {2}')
            print(f'Lama mean : {all_dict[i][j]["lama"][2].mean()}, std : {all_dict[i][j]["lama"][2].std()}, min : {all_dict[i][j]["lama"][2].min()}, max : {all_dict[i][j]["lama"][2].max()}')

            print(f'Label {i} - Label {j} _ Layer : {3}')
            print(f'Lama mean : {all_dict[i][j]["lama"][3].mean()}, std : {all_dict[i][j]["lama"][3].std()}, min : {all_dict[i][j]["lama"][3].min()}, max : {all_dict[i][j]["lama"][3].max()}')

            print(f'Label {i} - Label {j} _ Layer : {4}')
            print(f'Lama mean : {all_dict[i][j]["lama"][4].mean()}, std : {all_dict[i][j]["lama"][4].std()}, min : {all_dict[i][j]["lama"][4].min()}, max : {all_dict[i][j]["lama"][4].max()}')


Label 0 - Label 1 _ Layer : 0
Lama mean : 0.49104276299476624, std : 0.30104321241378784, min : -0.06061321496963501, max : 0.9483209848403931
Label 0 - Label 1 _ Layer : 1
Lama mean : 0.6206247806549072, std : 0.19752804934978485, min : 0.20239697396755219, max : 0.949982225894928
Label 0 - Label 1 _ Layer : 2
Lama mean : 0.6621878147125244, std : 0.20493528246879578, min : 0.22527503967285156, max : 0.9605312347412109
Label 0 - Label 1 _ Layer : 3
Lama mean : 0.6745672225952148, std : 0.209737166762352, min : 0.2252473682165146, max : 0.9623642563819885
Label 0 - Label 1 _ Layer : 4
Lama mean : 0.6963954567909241, std : 0.21580903232097626, min : 0.25123119354248047, max : 0.9677630662918091
Label 0 - Label 2 _ Layer : 0
Lama mean : 0.15634380280971527, std : 0.23554828763008118, min : -0.06872876733541489, max : 0.7742531299591064
Label 0 - Label 2 _ Layer : 1
Lama mean : 0.40887266397476196, std : 0.16453783214092255, min : 0.17391207814216614, max : 0.8375195264816284
Label 0 - La

## Test  Data

In [22]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [23]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [24]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.5121121406555176, std : 0.2874774932861328, min : -0.017242038622498512, max : 0.9495558738708496
Bert mean : -0.14266011118888855, std : 0.13258406519889832, min : -0.2987886369228363, max : 0.9136795997619629
Roberta mean : 0.05507628247141838, std : 0.19220171868801117, min : -0.08155005425214767, max : 0.8289508819580078

Label 0 - Label 2 :
Lama mean : 0.1943480521440506, std : 0.22652888298034668, min : -0.015423230826854706, max : 0.7862840890884399
Bert mean : -0.22013546526432037, std : 0.0641045793890953, min : -0.284465491771698, max : 0.1181434914469719
Roberta mean : -0.03148631006479263, std : 0.01759730465710163, min : -0.08271956443786621, max : 0.04119328409433365

Label 0 - Label 3 :
Lama mean : 0.17391426861286163, std : 0.2577849328517914, min : -0.05200731381773949, max : 0.8607800602912903
Bert mean : -0.14841324090957642, std : 0.22305595874786377, min : -0.3096223771572113, max : 0.9735559821128845
Roberta mean : 0.0084089515730

# llama layer별로 보기

In [ ]:
reps_dict = {i : {'lama' : {j : [] for j in range(6)},
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)